# Tutorial 1.2 - Spatial analysis with Python

```{attention}
Finnish university students are encouraged to use the CSC Notebooks platform.<br/>
<a href="https://notebooks.csc.fi/#/blueprint/c54303e865294208ba1ef381332fd69b"><img alt="CSC badge" src="https://img.shields.io/badge/launch-CSC%20notebook-blue.svg" style="vertical-align:text-bottom"></a>

Others can follow the lesson interactively using Binder. Check the rocket icon on the top of this page.
```

In the first week, we will take a quick tour to Python's (spatial) data science ecosystem and see how we can use some of the fundamental open source Python packages, such as:

 - pandas / geopandas
 - shapely
 - pysal
 - pyproj
 - osmnx / pyrosm
 - matplotlib (visualization)
 
As you can see, we won't use any GIS software for doing the programming (such as ArcGIS/arcpy or QGIS), but focus on learning the open source packages that are independent from any specific software. These libraries form nowadays not only the core for modern spatial data science, but they are also fundamental parts of commercial applications used and developed by many companies around the world. 

```{note} 

If you have experience working with the Python's spatial data science stack, this tutorial probably does not bring much new to you, but to get everyone on the same page, we will all go through this introductory tutorial.

```

**Contents:**

 - Reading / writing spatial data
 - Retrieving OpenStreetMap data
 - Reprojections
 - Spatial join
 - Plotting data with matplotlib

## Fundamental library: Geopandas

In this course, the most often used Python package that you will learn is [geopandas](https://geopandas.org/). Geopandas makes it possible to work with geospatial data in Python in a relatively easy way. Geopandas combines the capabilities of the data analysis library [pandas](https://pandas.pydata.org/pandas-docs/stable/) with other packages like [shapely](https://shapely.readthedocs.io/en/stable/manual.html) and [fiona](https://fiona.readthedocs.io/en/latest/manual.html) for managing spatial data. The main data structures in geopandas are `GeoSeries` and `GeoDataFrame` which extend the capabilities of `Series` and `DataFrames` from pandas. In case you wish to have additional help getting started with pandas, we recommend you to take a look lessons 5 and 6 from the openly available [Geo-Python -course](geo-python.github.io). The main difference between GeoDataFrames and pandas DataFrames is that a [GeoDataFrame](http://geopandas.org/data_structures.html#geodataframe) should contain (at least) one column for geometries. By default, the name of this column is `'geometry'`. The geometry column is a [GeoSeries](http://geopandas.org/data_structures.html#geoseries) which contains the geometries (points, lines, polygons, multipolygons etc.) as shapely objects. 

![geodataframe.png](img/geodataframe.png)


## Reading and writing spatial data

Next we will learn some of the basic functionalities of geopandas. We have a couple of GeoJSON files stored in the `data` folder that we will use.

We can read the data easily with `read_file()` -function:

In [ ]:
import geopandas as gpd

# Filepath
fp = "data/buildings.geojson"

# Read the file
data = gpd.read_file(fp)

# How does it look?
data.head()

As we can see, the GeoDataFrame contains various attributes in separate columns. The `geometry` column contains the spatial information. We can take a look of some of the basic information about our GeoDataFrame with command:

In [ ]:
data.info()

From here, we can see that our data is indeed a GeoDataFrame object with 486 entries and 34 columns. You can also get descriptive statistics of your data by calling:

In [ ]:
data.describe()

In this case, we didn't have many columns with numerical data, but typically you have numeric values in your dataset and this is a good way to get a quick view how the data look like.

Naturally, as the data is **spatial**, we want to visualize it to understand the nature of the data better. We can do this easily with `plot()` method:

In [ ]:
data.plot()

Now we can see that the data indeed represents buildings (in central Helsinki). Naturally we can also write this data to disk. Geopandas supports writing data to various data formats as well as to PostGIS which is the most widely used open source database for GIS. Let's write the data as a Shapefile to the same `data` directory from where we read the data. When writing data to local disk you can use `to_file()` method that exports the data in Shapefile format by default:

In [ ]:
# Output filepath
outfp = "data/buildings_copy.shp"
data.to_file(outfp)

## Retrieving data from OpenStreetMap

Now we have seen how to read spatial data from disk. OpenStreetMap (OSM) is probably the most well known and widely used spatial dataset/database in the world. Also in this course, we will use OSM data frequently. Hence, let's see how we can retrieve data from OSM using a library called [pyrosm](https://pyrosm.readthedocs.io/en/latest/). With `pyrosm` you can easily download and extract data from anywhere in the world based on OSM.PBF files that are distributed e.g. by [Geofabrik](http://download.geofabrik.de/). The tool aims to be an efficient way to parse OSM data covering large geographical areas (such as countries and cities), but as a downside, it is a bit limited in a sense how you can define your area of interest. With `pyrosm` you can extract OSM data from 654 regions in the world (covering all countries plus many city regions, see [docs for further info](https://pyrosm.readthedocs.io/en/latest/basics.html#available-datasets)).

```{note} 

In case you want to extract OSM data from smaller areas, e.g. using a buffer of 2 km around a specific location, we recommend using [OSMnx](https://github.com/gboeing/osmnx) library, which is more flexible in terms of specifying the area of interest. 

```

Let's see how we can download and extract OSM data for Helsinki Region using `pyrosm`:

In [ ]:
from pyrosm import OSM, get_data

# Download data for Helsinki
fp = get_data("helsinki")

# Initialize the reader object for Helsinki
osm = OSM(fp)

As a first step, we downloaded the data for "Helsinki" using the `get_data` function, which is a helper function that automates the data downloading process and stores the data locally in a temporary folder (inside `/tmp/` in this case). The next step that we did, was to initialize a reader object called `OSM`. The `OSM` takes the filepath to a given `osm.pbf` file as an input. Notice that at this point we didn't yet read any data into GeoDataFrame.

OSM is a "database of the world", hence it contains **a lot** of information about different things. With `pyrosm` you can easily extract information about:

- street networks --> `osm.get_network()`
- buildings --> `osm.get_buildings()`
- Points of Interest (POI) --> `osm.get_pois()`
- landuse --> `osm.get_landuse()`
- natural elements --> `osm.get_natural()`
- boundaries --> `osm.get_boundaries()`

Let's see how we can read all the buildings from Helsinki Region:

In [ ]:
buildings = osm.get_buildings()

In [ ]:
buildings.head()

Let's check how many buildings did we get:

In [ ]:
len(buildings)

Okay, so there are more than 150 thousand buildings in the Helsinki Region. Naturally, we would like to see them on a map. Let's plot our data using `plot()` (might take some time as there are many objects to plot):

In [ ]:
buildings.plot()

Great! As a result we got a map that seems to look correct. 

## Reprojecting data

As we can see from the previous maps that we have produced, the coordinates on the x and y axis hint that our geometries are represented in decimal degrees (WGS84). 
In many cases, you want to reproject your data to another CRS. Luckily, doing that is easy with `geopandas`. Let's first take a look what the Coordinate Reference System (CRS) of our GeoDataFrame is. We can access the CRS information of the GeoDataFrame by accessing an attribute called `crs`:

In [ ]:
buildings.crs

As a result, we get information about the CRS, and we can see that the data is indeed in WGS84. We can also see that the EPSG code for the CRS is 4326.
We can easily reproject our data by using a method `to_crs()`. The easiest way to use the method is to specify the destination CRS as an EPSG code. Let's reproject our data into EPSG 3067 which is the most widely used projected coordinate reference system used in Finland, EUREF-FIN: 

In [ ]:
projected = buildings.to_crs(epsg=3067)
projected.crs

As we can see, now we have an `Projected CRS` as a result. To confirm the difference, let's take a look at the geometry of the first row in our original `buildings` GeoDataFrame and the `projected` GeoDataFrame. To select a specific row in data, we can use the `loc` indexing:

In [ ]:
orig_geom = buildings.loc[0, "geometry"]
projected_geom = projected.loc[0, "geometry"]

print("Orig:\n", orig_geom, "\n")
print("Proj:\n", projected_geom)

As we can see the coordinates that form our Polygon has changed from decimal degrees to **meters**. Let's see what happens if we just call the geometries:

In [ ]:
orig_geom

In [ ]:
projected_geom

As you can see, we can draw the geometry directly in the screen, and we can easily see the difference in the shape of the two geometries. The `orig_geom` and `projected_geom` variables contain a Shapely geometry which is `Polygon` in this case. We can confirm this by checking the type:

In [ ]:
type(orig_geom)

These shapely geometries are used as the underlying data structure in most GIS packages in Python to present geometrical information. Shapely is fundamentally a Python wrapper for [GEOS](https://trac.osgeo.org/geos/) which is widely used library (written in C++) under the hood of many GIS softwares such as QGIS, GDAL, GRASS, PostGIS, Google Earth etc. Currently, there is [ongoing work](https://pygeos.readthedocs.io/en/latest/) to vectorize all the GEOS functionalities for Python and bring those eventually into Shapely which will greatly boost the performance of all geometry related operations in Python ecosystem (approaching the same efficiency as PostGIS). Some of these improvements can already be found under the hood of latest version of geopandas.

## Calculating area

One thing that is quite often interesting to know when working with spatial data, is the `area` of the geometries. In geopandas, we can easily calculate e.g. the area for each of our buildings by: 

In [ ]:
projected["building_area"] = projected.area
projected["building_area"].describe()

We calculated the area by calling `area` which is the attribute containing information about areas of the buildings measured based on the map units of the data. Hence, in this case because our data is projected in Euref-FIN the units that we stored in `"building_area"` column are **square meters**. It's important to always keep in mind the CRS when calculating areas, distances etc. with geometries.  

## Spatial join

A commonly needed GIS functionality, is to be able to merge information between two layers using location as the `key`. Hence, it is somewhat similar approach as *table join* but because the operation is based on geometries, it is called *spatial join*. 
Next, we will see how we can conduct a spatial join and merge information between two layers. We will read all restaurants from the OSM for Helsinki Region, and combine information from restaurants to the underlying building (*restaurants typically are within buildings*). We will again use `pyrosm` for reading the data, but this time we will use `get_pois()` function:

In [ ]:
# Read Points of Interest using the same OSM reader object that was initialized earlier
restaurants = osm.get_pois(custom_filter={"amenity": ["restaurant"]})
restaurants.plot()

In [ ]:
restaurants.info()

As we can see, the OSM for Helsinki Region contains 1388 restaurants altogether. As you can probably guess, the OSM data is far from "perfect" in terms of the quality of the restaurant listings. This is due to the voluntary nature of adding information to the OpenStreetMap, and the fact restaurants (as well as other POI features) are highly dynamic by nature, i.e. new amenities open and close all the time, and it is challenging to keep up to date with those changes (this is a challenge even for commercial companies).  

Joining data from buildings to the restaurants can be done easily using `sjoin()` function from geopandas:

In [ ]:
# Join information from buildings to restaurants
join = gpd.sjoin(restaurants, buildings)

# Print column names
print(join.columns)

# Show rows
join

In [ ]:
# Visualize the data as well
join.plot()

As we can see from the above, now we have merged information from the buildings to restaurants. The geometries of the `left` GeoDataFrame, i.e. restaurants were kept by default as the geometries.

### Selecting data using sjoin

One handy trick and efficient trick for spatial join is to use it for **selecting data**. We can e.g. select all buildings that intersect with restaurants by conducting the spatial join other way around, i.e. using the buildings as the left GeoDataFrame and the restaurants as the right GeoDataFrame:

In [ ]:
# Merge information from restaurants to buildings (conducts selection at the same time)
join2 = gpd.sjoin(buildings, restaurants, how="inner", op="intersects")
join2.plot()

As we can see (although the small building geometries are a bit poorly visible), the end result is a layer of buildings which intersected with the restaurants. This is a straightforward way to conduct simple spatial queries. You can specify with `op` parameter whether the binary predicate between the layers (i.e. the spatial relation between geometries) should be:

- `intersects`
- `contains`
- `within`

## Plotting data with matplotlib

Thus far, we haven't really made any effort to make our maps visually appealing. Let's next see how we can adjust the appearance of our map, and how we can visualize many layers on top of each other. Let's start by visualizing the buildings that we selected earlier and adjust a bit of the colors and figuresize. We can adjust the color of polygons with `facecolor` parameter and the figure size with `figsize` parameter that accepts a tuple of width and height as an argument:

In [ ]:
ax = join2.plot(facecolor="red", figsize=(12,12))

In [ ]:
join2.columns

Now with the bigger figure size, it is already a bit easier to see the selected buildings that have a restaurant inside them (according OSM). Let's color our buildings based on the building type. Hence, each building type category will receive a different color:  

In [ ]:
ax = join2.plot(column="building_left", cmap="RdYlBu", figsize=(12,12), legend=True)

Now we used the parameter `column` to specify the attribute that is used to specify the color for each building (can be categorical or continuous). We used `cmap` to specify the colormap for the categories and we added the legend by specifying `legend=True`. It is still a bit tricky to see what happens in our map. Luckily it is easy to **zoom in** to our map by using a specific commands (`set_xlim()` and `set_ylim()` that control the *axis* of our visualization:

In [ ]:
# Zoom into city center by specifying X and Y coordinate extent
# These values should be given in the units that our data is presented (here decimal degrees)
xmin, xmax = 24.92, 24.98
ymin, ymax = 60.15, 60.18

# Plot the map again
ax = join2.plot(column="building_left", cmap="RdYlBu", figsize=(12,12), legend=True)

# Control and set the x and y limits for the axis
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])

Now it is much easier to see how the building types are distributed in the city. To get a bit more context to our visualizaton. Let's also add roads with our buildings. 
To do that we first need to extract the roads from OSM:

In [ ]:
# Get roads (retrieves walkable roads by default)
roads = osm.get_network()

Now we can continue and add the roads as a layer to our visualization with gray line color:

In [ ]:
# Zoom into city center by specifying X and Y coordinate extent
# These values should be given in the units that our data is presented (here decimal degrees)
xmin, xmax = 24.92, 24.98
ymin, ymax = 60.15, 60.18

# Plot the map again
ax = join2.plot(column="building_left", cmap="RdYlBu", figsize=(12,12), legend=True)

# Plot the roads into the same axis
ax = roads.plot(ax=ax, edgecolor="gray", linewidth=0.75)

# Control and set the x and y limits for the axis
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])

Perfect! Now it is much easier to understand our map because the roads brought much more context (assuming you know Helsinki). We ware able to add the roads to the same map by specifying the `ax` parameter to point to the axis that we received when first plotting the `join2` (i.e. selected buildings). In a similar manner, you can add as many layers in your map as you wish. Let's still do a small visual trick and specify that the background color in our map is black instead of white. This can be done easily by changing the `style` of matplotlib visualization renderer:

In [ ]:
# Import matplotlib pyplot and use a dark_background theme
import matplotlib.pyplot as plt
plt.style.use("dark_background")

# Zoom into city center by specifying X and Y coordinate extent
# These values should be given in the units that our data is presented (here decimal degrees)
xmin, xmax = 24.92, 24.98
ymin, ymax = 60.15, 60.18

# Plot the map again
ax = join2.plot(column="building_left", cmap="RdYlBu", figsize=(12,12), legend=True)

# Plot the roads into the same axis
ax = roads.plot(ax=ax, edgecolor="gray", linewidth=0.75)

# Control and set the x and y limits for the axis
ax.set_xlim([xmin, xmax])
ax.set_ylim([ymin, ymax])

Awesome! Now we have a nice dark theme with our map. With this information you should be able to get going with Exercise 1. 

## Further information

For further information, we recommend checking the materials from [Automating GIS Processes](https://autogis.github.io/) -course (GIS things) and [Geo-Python](https://geo-python.github.io/) -course (intro to Python and data analysis with pandas). In addition, we always recommend to check the latest documentation from the websites of the libraries:

 - [geopandas](https://geopandas.org/) 
 - [pyrosm](https://pyrosm.readthedocs.io/en/latest/)
 - [matplotlib](https://matplotlib.org/)
 - [pandas](https://pandas.pydata.org/)